In [2]:
!git clone https://github.com/AminMohamed-3/Emotion-Classification.git
!pip install transformers dataset accelerate -q
import sys
sys.path.append("/kaggle/working/Emotion-Classification")

In [1]:
import os
import numpy as np
import torch
import wandb
from datasets import load_dataset
from dotenv import load_dotenv
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments,
)
import yaml
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
    NUM_LABELS = config["NUM_LABELS"
    ]
from Training.utils import compute_metrics
from Training.dataset import prepare_simplified_dataset, prepare_local_dataset

load_dotenv()

True

# Load & Tokenize dataset

In [2]:
model_checkpoint = "FacebookAI/roberta-base"  # Using a larger model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
dataset = prepare_local_dataset(tokenizer)

Filter:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

# Define Model

In [3]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification",
)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

KeyboardInterrupt: 

# Trainer

In [44]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,  # Training for longer
    per_device_train_batch_size=16,  # smaller batch size
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,  # Using a smaller LR
    save_total_limit=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_steps=20,
    weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type='cosine',
)

In [45]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
)

In [48]:
wandb.login(key=os.environ["wandb"])
trainer.train()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Epoch,Training Loss,Validation Loss
